In [ ]:
!pip install dowhy==0.12 econml==0.15
!pip install networkx==3.3

‼️Before running the next cell, press **Ctrl (Cmd) + M + .** and confirm that you want to restsrt the session.

Thank you 🙏🏼

In [ ]:
import dowhy
import econml
print(dowhy.__version__, econml.__version__)

In [ ]:
import warnings

import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt

from dowhy import CausalModel

warnings.simplefilter(
    action='ignore',
    category=FutureWarning
)

# The 4-Step Process of Causal Modeling



In [ ]:
# Read the data - dataset 1
df1 = pd.read_csv('https://raw.githubusercontent.com/AlxndrMlk/workshop-kbc-25/refs/heads/main/data/dataset1.csv')

## Step 1: Modeling the Structure

### 1.1 Define the Graph

In [ ]:
# Instantiate a grpah object
graph = nx.DiGraph()

In [ ]:
# Define edges
edges = [
    ('x1', 'y'),
    ('x2', 'x1'),
    ('x2', 'y'),
    ('x1', 'x3'),
    ('y', 'x3'),
]

# Encode the edges
graph.add_edges_from(edges)

# Optional: add unconnected nodes
graph.add_nodes_from(['x4', 'x5'])

In [ ]:
# Plot the graph
nx.draw(graph)

In [ ]:
# Improve the plot
plt.figure(figsize=(5, 3))
nx.draw_circular(
    graph,
    with_labels=True,
    node_size=400,
    font_color='white'
    )

### 1.2 Instantiate the DoWhy Model

In [ ]:
# Instantiate the DoWhy model
model = CausalModel(
    data=df1,
    graph=graph,
    treatment='x1',
    outcome='y',
)

In [ ]:
# View model
model.view_model(size=(5, 5))

## Step 2: Identify the Estimand

In [ ]:
estimand = model.identify_effect()
print(estimand)

## Step 3: Estimate the Causal Effect

In [ ]:
# Estimate the effect using linear regression
estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.linear_regression',
    target_units='ate')

print(f'Estimate of causal effect (linear regression): {estimate.value}')

## Step 4: Refute the Estimate

### 4.1 Add Random Common Cause

In [ ]:
refuter_1 = model.refute_estimate(
    estimand=estimand,
    estimate=estimate,
    method_name="random_common_cause"
)

print(refuter_1)

### 4.2 Placebo Treatment

In [ ]:
refuter_2 = model.refute_estimate(
    estimand=estimand,
    estimate=estimate,
    method_name="placebo_treatment_refuter",
    placebo_type="permute"
)

print(refuter_2)

### 4.3 Data Subset Refuter

In [ ]:
refuter_3 = model.refute_estimate(
    estimand=estimand,
    estimate=estimate,
    method_name="data_subset_refuter",
    subset_fraction=0.1
)

print(refuter_3)